In [ ]:
!pip install -q tflite-model-maker

     |████████████████████████████████| 616 kB 3.2 MB/s 
     |████████████████████████████████| 1.1 MB 45.7 MB/s 
     |████████████████████████████████| 120 kB 54.9 MB/s 
     |████████████████████████████████| 840 kB 39.4 MB/s 
     |████████████████████████████████| 87 kB 4.8 MB/s 
     |████████████████████████████████| 1.1 MB 44.8 MB/s 
     |████████████████████████████████| 1.2 MB 39.0 MB/s 
     |████████████████████████████████| 596 kB 54.5 MB/s 
     |████████████████████████████████| 77 kB 5.4 MB/s 
     |████████████████████████████████| 3.4 MB 45.7 MB/s 
     |████████████████████████████████| 6.4 MB 43.1 MB/s 
     |████████████████████████████████| 234 kB 45.8 MB/s 
     |████████████████████████████████| 25.3 MB 1.8 MB/s 
     |████████████████████████████████| 47.8 MB 55 kB/s 
     |████████████████████████████████| 99 kB 7.9 MB/s 
     |████████████████████████████████| 352 kB 53.5 MB/s 
     |████████████████████████████████| 462 kB 49.4 MB/s 
     |████████████████

In [ ]:
import os

import numpy as np

import tensorflow as tf
assert tf.__version__.startswith('2')

from tflite_model_maker import model_spec
from tflite_model_maker import image_classifier
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.image_classifier import DataLoader

import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
image_path = "/content/gdrive/MyDrive/ImageClassificationEIS/"
#image_path = os.path.join(os.path.dirname(image_path), 'data')
print(image_path)

/content/gdrive/MyDrive/ImageClassificationEIS/


In [ ]:
data = DataLoader.from_folder(image_path)
print(data)

INFO:tensorflow:Load image with size: 5015, num_label: 5, labels: road, shop, sign, trafficsignal, tvmonitor.


In [ ]:
train_data, rest_data = data.split(0.9)
val_data, test_data = rest_data.split(0.5)
print(train_data,val_data)

<tensorflow_examples.lite.model_maker.core.data_util.image_dataloader.ImageClassifierDataLoader object at 0x7f1e7cd547d0> <tensorflow_examples.lite.model_maker.core.data_util.image_dataloader.ImageClassifierDataLoader object at 0x7f1e7024cbd0>


In [ ]:
plt.figure(figsize=(10,10))
for i, (image, label) in enumerate(data.gen_dataset().unbatch().take(25)):
  plt.subplot(5,5,i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(image.numpy(), cmap=plt.cm.gray)
  plt.xlabel(data.index_to_label[label.numpy()])
plt.show()

In [ ]:
inception_v4_spec = image_classifier.ModelSpec(
    uri='https://tfhub.dev/google/imagenet/inception_v4/feature_vector/1')
inception_v4_spec.input_image_shape = [299, 299]

In [ ]:
model = image_classifier.create(train_data, model_spec=model_spec.get(inception_v3_spec), validation_data=val_data,epochs=1)

INFO:tensorflow:Retraining the models...


INFO:tensorflow:Retraining the models...


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hub_keras_layer_v1v2_1 (Hub  (None, 2048)             21802784  
 KerasLayerV1V2)                                                 
                                                                 
 dropout_1 (Dropout)         (None, 2048)              0         
                                                                 
 dense_1 (Dense)             (None, 5)                 10245     
                                                                 
Total params: 21,813,029
Trainable params: 10,245
Non-trainable params: 21,802,784
_________________________________________________________________
None


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


141/141 [==============================] - 1016s 7s/step - loss: 0.5756 - accuracy: 0.9328 - val_loss: 0.4904 - val_accuracy: 0.9681


In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hub_keras_layer_v1v2_1 (Hub  (None, 2048)             21802784  
 KerasLayerV1V2)                                                 
                                                                 
 dropout_1 (Dropout)         (None, 2048)              0         
                                                                 
 dense_1 (Dense)             (None, 5)                 10245     
                                                                 
Total params: 21,813,029
Trainable params: 10,245
Non-trainable params: 21,802,784
_________________________________________________________________





Evaluate 

In [ ]:
loss, accuracy = model.evaluate(test_data)

8/8 [==============================] - 67s 7s/step - loss: 0.5027 - accuracy: 0.9681


In [ ]:
# A helper function that returns 'red'/'black' depending on if its two input
# parameter matches or not.
def get_label_color(val1, val2):
  if val1 == val2:
    return 'black'
  else:
    return 'red'

# Then plot 100 test images and their predicted labels.
# If a prediction result is different from the label provided label in "test"
# dataset, we will highlight it in red color.
plt.figure(figsize=(20, 20))
predicts = model.predict_top_k(test_data)
for i, (image, label) in enumerate(test_data.gen_dataset().unbatch().take(100)):
  ax = plt.subplot(10, 10, i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(image.numpy(), cmap=plt.cm.gray)

  predict_label = predicts[i][0][0]
  color = get_label_color(predict_label,
                          test_data.index_to_label[label.numpy()])
  ax.xaxis.label.set_color(color)
  plt.xlabel('Predicted: %s' % predict_label)
plt.show()

In [ ]:
model.export(export_dir='.')

INFO:tensorflow:Assets written to: /tmp/tmp8l3242fj/assets


INFO:tensorflow:Assets written to: /tmp/tmp8l3242fj/assets
/usr/local/lib/python3.7/dist-packages/tensorflow/lite/python/convert.py:746: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


INFO:tensorflow:Label file is inside the TFLite model with metadata.


INFO:tensorflow:Label file is inside the TFLite model with metadata.


INFO:tensorflow:Saving labels in /tmp/tmpap0eoih9/labels.txt


INFO:tensorflow:Saving labels in /tmp/tmpap0eoih9/labels.txt


INFO:tensorflow:TensorFlow Lite model exported successfully: ./model.tflite


INFO:tensorflow:TensorFlow Lite model exported successfully: ./model.tflite


In [ ]:
model.evaluate_tflite('model.tflite', test_data)